# Introduction

In this notebook, we will train BERT model on the IMDB Movie Review Classification dataset.

## Setup

In [1]:
!pip install -U transformers datasets evaluate accelerate
!pip install scikit-learn

## Imports

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    DataCollatorWithPadding,
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    pipeline
)

import evaluate
import numpy as np

## Hyperparameters

In [3]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 2

## Download the Dataset

In [4]:
train_dataset = load_dataset("imdb", split='train')
test_dataset = load_dataset("imdb", split='test')

In [5]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [6]:
# Visualize a sample.
train_dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Dataset Information

In [7]:
id2label = {
    0: "Negative",
    1: "Positive",
}
label2id = {
    "Negative": 0,
    "Positive": 1,
}

## Tokenize the Dataset

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
# Helper function for preprocessing.
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_train = train_dataset.map(
    preprocess_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

In [11]:
tokenized_test = test_dataset.map(
    preprocess_function, 
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

In [12]:
# Initialize data collator.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluation Metrics

In [13]:
accuracy = evaluate.load('accuracy')

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Model

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2, 
    id2label=id2label, 
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

109,483,778 total parameters.
109,483,778 training parameters.


## Training Arguments

In [17]:
training_args = TrainingArguments(
    output_dir="imdb_classification",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

## Training

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [19]:
history = trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.272400,0.212047,0.924360
2,0.097800,0.188838,0.943120


In [20]:
print(history.global_step)

1564


In [21]:
model = AutoModelForSequenceClassification.from_pretrained(f"imdb_classification/checkpoint-{history.global_step}")
# model = AutoModelForSequenceClassification.from_pretrained(f"arxiv_classification/checkpoint-1332")

In [22]:
classify = pipeline(task='text-classification', model=model, tokenizer=tokenizer)

In [23]:
file = open('inference_data/text1.txt') 
content = file.read()
print(content)

A grim, gritty, and gripping super-noir, The Batman ranks among the Dark Knight's bleakest -- and most thrillingly ambitious -- live-action outings.


In [24]:
classify(content)

[{'label': 'Positive', 'score': 0.9977442026138306}]